In [1]:
from dwave.system.samplers import DWaveSampler
from itertools import groupby, product
import math
import random

In [2]:
sampler = DWaveSampler()
couplers = set([tuple(coupler) for coupler in sampler.properties['couplers']])
H_range = sampler.properties['h_range']
J_range = sampler.properties['j_range']
H_range, J_range

([-2.0, 2.0], [-1.0, 1.0])

In [19]:
cell_couplers = [(i, j) for i, j in product(range(0, 4), range(4, 8))]
cell_H = dict((i, random.uniform(H_range[0], H_range[1]) / 2) for i in range(8))
cell_J = dict(((i, j), random.uniform(J_range[0], J_range[1])) for i, j in cell_couplers)
cell_H, cell_J

({0: 0.08370421937327488,
  1: -0.4262382159509144,
  2: -0.8442658033776838,
  3: 0.8529403860828175,
  4: -0.4582872907843023,
  5: 0.9397036067742746,
  6: 0.45201076240423466,
  7: -0.21887130414000278},
 {(0, 4): -0.6484448705158581,
  (0, 5): 0.8495677682108309,
  (0, 6): 0.0814170700363055,
  (0, 7): -0.8826579310529952,
  (1, 4): 0.02360829850414259,
  (1, 5): 0.5835720017483192,
  (1, 6): -0.12307374659144665,
  (1, 7): 0.5263749028358928,
  (2, 4): -0.5550887576431878,
  (2, 5): 0.12720833356641892,
  (2, 6): 0.9017010972965778,
  (2, 7): 0.4040579195696532,
  (3, 4): -0.9524824027242849,
  (3, 5): -0.17834159922350623,
  (3, 6): -0.6865360428305183,
  (3, 7): -0.10872830759878704})

In [20]:
active_cells = list(filter(lambda c: set([(i + 8 * c, j + 8 * c) for i, j in cell_couplers]).issubset(couplers), range(256)))
len(active_cells)

244

In [21]:
H = dict((i + 8 * c, v) for i, v in cell_H.items() for c in active_cells)
J = dict(((i + 8 * c, j + 8 * c), v) for (i, j), v in cell_J.items() for c in active_cells)
len(H), len(J)

(1952, 3904)

In [22]:
def energy(spins):
    return sum([v * spins[i] for i, v in cell_H.items()] + [v * spins[i] * spins[j] for (i, j), v in cell_J.items()])

In [26]:
response = sampler.sample_ising(H, J, num_reads=100, answer_mode='raw', auto_scale=False)

In [30]:
res_hist = [(spins, sum(1 for _ in group), energy(spins)) for spins, group in
 groupby(sorted(tuple([sample[i + 8 * c] for i in range(8)]) for c in active_cells for sample, _, _ in response.data()))]
num_sum = sum(num for _, num, _ in res_hist)
res_normed_hist = [(spins, num / num_sum, energy) for spins, num, energy in res_hist]

print(num_sum)
list(sorted(res_normed_hist, key=lambda x: x[2]))

24400


[((1, 1, 1, -1, 1, -1, -1, 1), 0.6146721311475409, -7.464823044441264),
 ((1, 1, 1, -1, 1, -1, -1, -1), 0.15811475409836065, -7.340086834063936),
 ((-1, 1, 1, -1, -1, -1, -1, -1), 0.2265983606557377, -7.3362707640240785),
 ((1, -1, 1, -1, 1, -1, -1, 1), 0.0004098360655737705, -6.791316504905762),
 ((1, 1, 1, -1, -1, -1, -1, 1), 0.00012295081967213115, -6.093362609011422),
 ((-1, 1, 1, -1, -1, 1, -1, -1), 8.19672131147541e-05, -5.377755217820702)]

In [32]:
inv_temp = 10
all_hist = [(spins, math.exp(-energy(spins) * inv_temp), energy(spins)) for spins in product([-1, 1], repeat=8)]
p_sum = sum(p for _, p, _ in all_hist)
all_normed_hist = [(spins, p / p_sum, energy) for spins, p, energy in all_hist]
sorted(all_normed_hist, key=lambda x: x[2])[:20]

[((1, 1, 1, -1, 1, -1, -1, 1), 0.6389936314762205, -7.464823044441264),
 ((1, 1, 1, -1, 1, -1, -1, -1), 0.183558310271359, -7.340086834063936),
 ((-1, 1, 1, -1, -1, -1, -1, -1), 0.17668556498410257, -7.3362707640240785),
 ((1, -1, 1, -1, 1, -1, -1, 1), 0.0007594423178072515, -6.791316504905762),
 ((1, 1, 1, 1, 1, -1, -1, 1), 1.2655456975753578e-06, -6.1516084088137255),
 ((-1, 1, 1, -1, 1, -1, -1, -1), 8.684341612608302e-07, -6.1139517173904885),
 ((1, 1, 1, -1, -1, -1, -1, 1), 7.068374268688657e-07, -6.093362609011422),
 ((1, 1, 1, -1, -1, -1, -1, -1), 2.0304722507618733e-07, -5.968626398634093),
 ((1, 1, 1, 1, 1, -1, -1, -1), 4.6962498309117566e-09, -5.591958968041247),
 ((1, -1, 1, 1, 1, -1, -1, 1), 1.504097866573773e-09, -5.478101869278223),
 ((-1, 1, 1, -1, -1, 1, -1, -1), 5.514118883607267e-10, -5.377755217820702),
 ((1, -1, 1, -1, -1, -1, -1, 1), 3.267380027967145e-10, -5.325422875459351),
 ((1, 1, 1, 1, 1, -1, 1, 1), 4.667535313837454e-12, -4.9005701281834195),
 ((-1, -1, 1, -1

In [34]:
!pip list | grep dwave

dwave-cloud-client     0.4.16    
dwave-neal             0.4.2     
dwave-networkx         0.6.6     
dwave-ocean-sdk        1.0.3     
dwave-qbsolv           0.2.9     
dwave-system           0.5.4     
dwavebinarycsp         0.0.9     
